In [1]:
import mne
from mne.datasets import eegbci
from mne.io import read_raw_edf
import pandas as pd
import numpy as np

##########
subjects = range(1,110)
runs = [1,2] # Baseline: Eyes open vs eyes closed

raw_fnames = list()
for subject in subjects: 
    raw_fnames.append(eegbci.load_data(subject, runs))

# We now need to flatten the list
flat_list = [item for sublist in raw_fnames for item in sublist]

pd.set_option('display.max_colwidth', -1)

df = pd.DataFrame(raw_fnames)
df = df.rename(columns={0: "Eyes_Open", 1: "Eyes_Closed"})

raw_files_open = [read_raw_edf(f, preload=True, stim_channel='auto') for f in df.Eyes_Open]
raw_files_closed = [read_raw_edf(f, preload=True, stim_channel='auto') for f in df.Eyes_Closed]

# The events are spaced evenly every 1 second.
duration = 1.
event_id_o = 1 # Eyes open
event_id_c = 0 # Eyes closed

# for fixed size events no start time before and after event
tmin = 0.
tmax = 0.99  # inclusive tmax, 1 second epochs


# create a fixed size events array
# start=0 and stop=None by default


# Get all the open events


temp_o = []

for raw_ in raw_files_open:    
    events_o = mne.make_fixed_length_events(raw_, event_id_o, duration=duration)
    # create :class:`Epochs <mne.Epochs>` object
    epochs_o = mne.Epochs(raw_, events=events_o, event_id=event_id_o, tmin=tmin,
                        tmax=tmax, baseline=None, verbose=True)
    for ep in epochs_o:
        temp_o.append(ep)

big_raw_o = np.array(temp_o)

# Get all the open events


temp_c = []

for raw_ in raw_files_closed:    
    events_c = mne.make_fixed_length_events(raw_, event_id_c, duration=duration)
    # create :class:`Epochs <mne.Epochs>` object
    epochs_c = mne.Epochs(raw_, events=events_c, event_id=event_id_c, tmin=tmin,
                        tmax=tmax, baseline=None, verbose=True)
    for ep in epochs_c:
        temp_c.append(ep)

big_raw_c = np.array(temp_c)

labels_o = np.ones(len(big_raw_o))
labels_c = np.zeros(len(big_raw_c))

big_raw = np.concatenate((big_raw_o[:, :-1, :], big_raw_c[:, :-1, :]), axis=0)
big_labels = np.concatenate((labels_o, labels_c), axis=0)

Extracting edf Parameters from /Users/axeluran/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Created Raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Ready.
Extracting edf Parameters from /Users/axeluran/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S002/S002R01.edf...
EDF file detected
Setting channel info structure...
Created Raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Ready.
Extracting edf Parameters from /Users/axeluran/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S003/S003R01.edf...
EDF file detected
Setting channel info structure...
Created Raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Ready.
Extracting edf Parameters from /Users/axeluran/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S004/S004R01.edf...
EDF file detected
Setting channel info structure...
Created Raw.info structure...
Re

In [8]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Initialization of names and classifiers

# linear will not be good enough

h = .02  # step size in the mesh

names = ["RBF SVM", "Random Forest", "Neural Net"]

classifiers = [
    SVC(gamma=2, C=1, verbose=True),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, verbose=True),
    MLPClassifier(alpha=1, verbose=True)]

In [9]:
# preprocess dataset, split into training and test part
X, y = big_raw, big_labels
nsamples, nx, ny = X.shape
X_d2 = X.reshape((nsamples,nx*ny))
X_d2 = StandardScaler().fit_transform(X_d2)
X_train, X_test, y_train, y_test = \
    train_test_split(X_d2, y, test_size=.4, random_state=42)

In [10]:
# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    
    print('Classifer: ' + str(name) + '\n' + 'Score: ' + str(score))

[LibSVM]Classifer: RBF SVM
Score: 0.49943587815


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Classifer: Random Forest
Score: 0.605866867243
Iteration 1, loss = 2.20546007
Iteration 2, loss = 1.11110176
Iteration 3, loss = 0.94833458
Iteration 4, loss = 0.84974514
Iteration 5, loss = 0.74114575
Iteration 6, loss = 0.73400395
Iteration 7, loss = 0.62637256
Iteration 8, loss = 0.68169392
Iteration 9, loss = 0.59790146
Iteration 10, loss = 0.55588173
Iteration 11, loss = 0.47548854
Iteration 12, loss = 0.51564286
Iteration 13, loss = 0.46177267
Iteration 14, loss = 0.47038903
Iteration 15, loss = 0.45512063
Iteration 16, loss = 0.46424535
Iteration 17, loss = 0.39676483
Iteration 18, loss = 0.36881022
Iteration 19, loss = 0.37177575
Iteration 20, loss = 0.39513795
Iteration 21, loss = 0.41511590
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Classifer: Neural Net
Score: 0.738811583302
